In [1]:
import math
import numpy as np
import wandb
import pickle
import os
import shapely.wkt as wkt
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
from torch_geometric.transforms import LineGraph

import gzip
import xml.etree.ElementTree as ET

import torch
import torch_geometric
from torch_geometric.data import Data

import processing_io as pio
import sys
import os
import joblib
import json

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from shapely.geometry import Point, LineString, box
from matplotlib.colors import TwoSlopeNorm

from shapely.ops import unary_union
from mpl_toolkits.axes_grid1 import make_axes_locatable
from torch_geometric.data import Data, Batch
import torch
from torch_geometric.data import Data
import help_functions as hf

districts = gpd.read_file("../../data/visualisation/districts_paris.geojson")


# Get the current working directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from gnn_architecture import MyGnn  # or whatever you need to import
import help_functions as hf
import gnn_io as gio
from enum import IntEnum

# highway_mapping = {
#     'trunk': 0, 'trunk_link': 0, 'motorway_link': 0,
#     'primary': 1, 'primary_link': 1,
#     'secondary': 2, 'secondary_link': 2,
#     'tertiary': 3, 'tertiary_link': 3,
#     'residential': 4, 'living_street': 5,
#     'pedestrian': 6, 'service': 7,
#     'construction': 8, 'unclassified': 9,
#     'np.nan': -1
# }

def compute_r2_torch_with_mean_targets(mean_targets, preds, targets):
    ss_tot = torch.sum((targets - mean_targets) ** 2)
    ss_res = torch.sum((targets - preds) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2

import math
import numpy as np
import wandb
import pickle
import os
import shapely.wkt as wkt
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
from torch_geometric.transforms import LineGraph

import gzip
import xml.etree.ElementTree as ET

import torch
import torch_geometric
from torch_geometric.data import Data

import processing_io as pio
import sys
import os
import joblib
import json

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from shapely.geometry import Point, LineString, box
from matplotlib.colors import TwoSlopeNorm

from shapely.ops import unary_union
from mpl_toolkits.axes_grid1 import make_axes_locatable
from torch_geometric.data import Data, Batch
import torch
from torch_geometric.data import Data
import alphashape
from matplotlib.lines import Line2D
from shapely.geometry import Polygon

In [2]:
run_path = '/home/enatterer/Development/gnn_predicting_effects_of_traffic_policies/data/runs_21_10_2024/'
point_net_conv_layer_structure_local_mlp = "64,128"
point_net_conv_layer_structure_global_mlp = "256,64"
gat_conv_layer_structure = "128,256,512,256"
dropout = 0.3
use_dropout = False 
predict_mode_stats = False
in_channels = 13
out_channels = 1 

pnc_l_string = point_net_conv_layer_structure_local_mlp.replace(',', '_')
pnc_g_string = point_net_conv_layer_structure_global_mlp.replace(',', '_') 
gat_string = gat_conv_layer_structure.replace(',', '_')

unique_model_description = f"pnc_local_[{pnc_l_string}]_" + \
f"pnc_global_[{pnc_g_string}]_" + \
f"gat_conv_[{gat_string}]_" + \
f"use_dropout_{use_dropout}_" + \
f"dropout_{dropout}_" + \
f"predict_mode_stats_{predict_mode_stats}" + "/"
        
run_path = run_path + unique_model_description

base_case_path = '../../data/test_data/pop_1pm_basecase_mean_links_NEW.geojson'
links_base_case = gpd.read_file(base_case_path, crs="EPSG:4326")
    
model_path = run_path +  'trained_model/model.pth'
data_created_during_training = run_path + 'data_created_during_training/'

scaler_x = joblib.load(data_created_during_training + 'test_x_scaler.pkl')
scaler_pos = joblib.load(data_created_during_training + 'test_pos_scaler.pkl')

StandardScaler()

In [4]:
# Load the test dataset created during training
test_set_dl = torch.load(data_created_during_training + 'test_dl.pt')

# Load the DataLoader parameters
with open(data_created_during_training + 'test_loader_params.json', 'r') as f:
    test_set_dl_loader_params = json.load(f)
    
# Remove or correct collate_fn if it is incorrectly specified
if 'collate_fn' in test_set_dl_loader_params and isinstance(test_set_dl_loader_params['collate_fn'], str):
    del test_set_dl_loader_params['collate_fn']  # Remove it to use the default collate function
    
test_set_loader = torch.utils.data.DataLoader(test_set_dl, **test_set_dl_loader_params)

In [5]:
test_set_loader.dataset[0].x

tensor([[-0.4045, -0.8099, -0.7885,  ...,  5.5740,  4.7530, -0.2053],
        [-0.4045, -0.8099, -0.7885,  ...,  5.5740,  4.7530, -0.2053],
        [-0.4045, -0.8099, -0.7885,  ...,  5.5740,  4.7530, -0.2053],
        ...,
        [-0.4045, -0.8099, -0.7885,  ..., -0.1794, -0.2104,  4.8717],
        [-0.4045, -0.8099, -0.7885,  ..., -0.1794, -0.2104, -0.2053],
        [-0.4045, -0.8099, -0.7885,  ..., -0.1794, -0.2104, -0.2053]])

In [6]:
scaler_x.inverse_transform(scaler_x.inverse_transform(test_set_loader.dataset[0].x)[0].reshape(1, -1))[0]


array([ 5.91181837e+00,  1.02575140e+03,  9.57126446e+02, -6.85745560e+01,
        8.11514268e+00,  1.12639089e+01,  2.30213077e+03,  8.91361584e-01,
        4.60725755e-01,  9.46857241e-01,  2.04990446e-01,  2.43864672e-01,
        4.04303145e-02])

In [40]:
len(test_set_loader.dataset)

497

In [41]:
point_net_conv_layer_structure_local_mlp = [int(x) for x in point_net_conv_layer_structure_local_mlp.split(',')]
point_net_conv_layer_structure_global_mlp = [int(x) for x in point_net_conv_layer_structure_global_mlp.split(',')]
gat_conv_layer_structure = [int(x) for x in gat_conv_layer_structure.split(',')]

print("point_net_conv_layer_structure_local_mlp:", point_net_conv_layer_structure_local_mlp)
print("point_net_conv_layer_structure_global_mlp:", point_net_conv_layer_structure_global_mlp)
print("gat_conv_layer_structure:", gat_conv_layer_structure)

point_net_conv_layer_structure_local_mlp: [64, 128]
point_net_conv_layer_structure_global_mlp: [256, 64]
gat_conv_layer_structure: [128, 256, 512, 256]


In [42]:
model = MyGnn(in_channels=in_channels, out_channels=out_channels, 
                    point_net_conv_layer_structure_local_mlp=point_net_conv_layer_structure_local_mlp, 
                    point_net_conv_layer_structure_global_mlp = point_net_conv_layer_structure_global_mlp,
                    gat_conv_layer_structure=gat_conv_layer_structure,
                    dropout=dropout,
                    use_dropout=use_dropout, 
                    predict_mode_stats=predict_mode_stats)

# Load the model state dictionary
model.load_state_dict(torch.load(model_path))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Model initialized
MyGnn(
  (point_net_conv_1): PointNetConv(local_nn=Sequential(
    (0): Linear(in_features=15, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
  ), global_nn=Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): ReLU()
  ))
  (point_net_conv_2): PointNetConv(local_nn=Sequential(
    (0): Linear(in_features=66, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
  ), global_nn=Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): ReLU()
  ))
  (point_net_conv_3): PointNetConv(local_nn=Sequential(
    (0)

/opt/anaconda3/envs/chenhao-gnn/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [43]:
def validate_model_on_test_set(model, dataset, loss_func, device):
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_targets = []
    
    with torch.inference_mode():
        if isinstance(dataset, list):
            for data in dataset:
                input_node_features, targets = data.x.to(device), data.y.to(device)
                predicted = model(data.to(device))
                loss = loss_func(predicted, targets).item()
                total_loss += loss
                all_preds.append(predicted)
                all_targets.append(targets)
        else:
            input_node_features, targets = dataset.x.to(device), dataset.y.to(device)
            predicted = model(dataset.to(device))
            loss = loss_func(predicted, targets).item()
            total_loss += loss
            all_preds.append(predicted)
            all_targets.append(targets)
    
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    
    mean_targets = torch.mean(all_targets)
    r_squared = compute_r2_torch_with_mean_targets(mean_targets=mean_targets, preds=all_preds, targets=all_targets)
    baseline_loss = loss_func(all_targets, torch.full_like(all_preds, mean_targets))
    
    avg_loss = total_loss / len(dataset)
    
    return avg_loss, r_squared, all_targets, all_preds, baseline_loss

# Example usage:
loss_fct = torch.nn.MSELoss()
test_loss, r_squared, actual_vals, predictions, baseline_loss = validate_model_on_test_set(model, test_set_loader.dataset, loss_fct, device)

print(f"Test Loss: {test_loss}")
print(f"R-squared: {r_squared}")
print(f"Baseline Loss: {baseline_loss}")

Test Loss: 1.60782257963714
R-squared: 0.44928133487701416
Baseline Loss: 2.919499158859253


In [44]:
my_test_x = test_set_loader.dataset[0].x
my_test_x = my_test_x.to('cpu')

In [66]:
my_test_x[0]

tensor([-0.4045, -0.8099, -0.7885,  0.2366, -2.0563,  0.6009, -0.6492, -2.8644,
        -0.9243,  1.1068,  5.5740,  4.7530, -0.2053])

In [70]:
print(scaler_x.scale_)
print(scaler_x.mean_)
print(scaler_x.var_)

[1.46146918e+01 1.26642926e+03 1.21392798e+03 2.89878194e+02
 3.94645961e+00 2.13702735e+00 1.10319280e+02 3.11184999e-01
 4.98455147e-01 4.97435285e-01 1.73808681e-01 2.01475470e-01
 1.96966252e-01]
[ 5.91181886e+00  1.02571066e+03  9.57135617e+02 -6.85750444e+01
  8.11514308e+00  2.71579961e+00  9.16523221e+01  8.91361593e-01
  4.60725755e-01  4.49421965e-01  3.11817598e-02  4.23892100e-02
  4.04303147e-02]
[2.13589215e+02 1.60384307e+06 1.47362113e+06 8.40293671e+04
 1.55745435e+01 4.56688589e+00 1.21703436e+04 9.68361037e-02
 2.48457534e-01 2.47441862e-01 3.02094577e-02 4.05923649e-02
 3.87957044e-02]


In [82]:
scaler_x.inverse_transform(scaler_x.inverse_transform(my_test_x)[0].reshape(1, -1))[0]


array([ 5.91181837e+00,  1.02575140e+03,  9.57126446e+02, -6.85745560e+01,
        8.11514268e+00,  1.12639089e+01,  2.30213077e+03,  8.91361584e-01,
        4.60725755e-01,  9.46857241e-01,  2.04990446e-01,  2.43864672e-01,
        4.04303145e-02])

In [77]:
inversed_values = scaler_x.inverse_transform(my_test_x)[0]

In [80]:
scaler_x.transform(inversed_values.reshape(1, -1))[0]

array([-0.40451205, -0.80992335, -0.78846163,  0.23656504, -2.0563097 ,
        0.60092837, -0.64916325, -2.86441064, -0.92430735,  1.10683346,
        5.57404995,  4.75298929, -0.20526519])

In [78]:
for i in range(len(inversed_values)):
    original_value = inversed_values[i] * scaler_x.scale_[i] + scaler_x.mean_[i]
    print(f"Original value at index {i}: {original_value}")

Original value at index 0: 5.911818374951527
Original value at index 1: 1025.751403219784
Original value at index 2: 957.1264463457123
Original value at index 3: -68.57455599426434
Original value at index 4: 8.115142678632898
Original value at index 5: 11.263908940781429
Original value at index 6: 2302.1307728289835
Original value at index 7: 0.8913615836835325
Original value at index 8: 0.4607257548492381
Original value at index 9: 0.9468572407759313
Original value at index 10: 0.20499044624495127
Original value at index 11: 0.24386467170578763
Original value at index 12: 0.0404303144672288


In [47]:
test_loss, r_squared, actual_vals, predictions, baseline_loss = validate_model_on_test_set(model, test_set_loader.dataset[0], loss_fct, device)

print(f"Test Loss: {test_loss}")
print(f"R-squared: {r_squared}")
print(f"Baseline Loss: {baseline_loss}")

Test Loss: 0.22439918915430704
R-squared: 0.5001441240310669
Baseline Loss: 2.6935665607452393


In [48]:
my_data = test_set_loader.dataset[0]

In [49]:
from enum import IntEnum

class EdgeFeatures(IntEnum):
    VOL_BASE_CASE = 0
    CAPACITY_BASE_CASE = 1
    CAPACITIES_NEW = 2
    CAPACITY_REDUCTION = 3
    FREESPEED = 4
    HIGHWAY = 5
    LENGTH = 6
    ALLOWED_MODE_CAR = 7
    ALLOWED_MODE_BUS = 8
    ALLOWED_MODE_PT = 9
    ALLOWED_MODE_TRAIN = 10
    ALLOWED_MODE_RAIL = 11
    ALLOWED_MODE_SUBWAY = 12
    
def data_to_geodataframe(data, original_gdf, predicted_values):
    # Extract the edge index and node features
    node_features = data.x.cpu().numpy()
    target_values = data.y.cpu().numpy()
    predicted_values = predicted_values.cpu().numpy() if isinstance(predicted_values, torch.Tensor) else predicted_values
    
    edge_data = {
        'from_node': original_gdf["from_node"].values,
        'to_node': original_gdf["to_node"].values,
        'vol_base_case': node_features[:, EdgeFeatures.VOL_BASE_CASE],  
        'capacity_base_case': node_features[:, EdgeFeatures.CAPACITY_BASE_CASE],  
        'capacities_new': node_features[:, EdgeFeatures.CAPACITIES_NEW],  
        'capacity_reduction': node_features[:, EdgeFeatures.CAPACITY_REDUCTION],  
        'freespeed': node_features[:, EdgeFeatures.FREESPEED],  
        'highway': node_features[:, EdgeFeatures.HIGHWAY],
        'length': node_features[:, EdgeFeatures.LENGTH],
        'allowed_mode_car': node_features[:, EdgeFeatures.ALLOWED_MODE_CAR],
        'allowed_mode_bus': node_features[:, EdgeFeatures.ALLOWED_MODE_BUS],
        'allowed_mode_pt': node_features[:, EdgeFeatures.ALLOWED_MODE_PT],
        'allowed_mode_train': node_features[:, EdgeFeatures.ALLOWED_MODE_TRAIN],
        'allowed_mode_rail': node_features[:, EdgeFeatures.ALLOWED_MODE_RAIL],
        'allowed_mode_subway': node_features[:, EdgeFeatures.ALLOWED_MODE_SUBWAY],
        'vol_car_change_actual': target_values.squeeze(),  
        'vol_car_change_predicted': predicted_values.squeeze()
    }
    # Convert to DataFrame
    edge_df = pd.DataFrame(edge_data)
    # Create LineString geometry
    edge_df['geometry'] = original_gdf["geometry"].values
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(edge_df, geometry='geometry')
    return gdf

gdf = data_to_geodataframe(data=my_data, original_gdf=links_base_case, predicted_values=predictions)

def map_to_original_values(input_gdf: gpd.GeoDataFrame, scaler_x):
    gdf_og_values = input_gdf.copy()
    # gdf_og_values['og_vol_car_change_actual'] = gdf['vol_car_change_actual']
    # gdf_og_values['og_vol_car_change_predicted'] = gdf['vol_car_change_predicted']
    
    edge_feature_columns = [feature.name.lower() for feature in EdgeFeatures]
    original_values = scaler_x.inverse_transform(gdf_og_values[edge_feature_columns])
    
    
    
    gdf_og_values[edge_feature_columns] = original_values
    return gdf_og_values

# gdf_with_og_values = map_to_original_values(input_gdf=gdf, scaler_x=scaler_x)
# gdf_with_og_values.head()

In [50]:
edge_feature_columns = [feature.name.lower() for feature in EdgeFeatures]

In [51]:
gdf[edge_feature_columns]

,vol_base_case,capacity_base_case,capacities_new,capacity_reduction,freespeed,highway,length,allowed_mode_car,allowed_mode_bus,allowed_mode_pt,allowed_mode_train,allowed_mode_rail,allowed_mode_subway
0,-0.404512,-0.809923,-0.788462,0.236565,-2.05631,0.600928,-0.649163,-2.864411,-0.924307,1.106833,5.574050,4.752989,-0.205265
1,-0.404512,-0.809923,-0.788462,0.236565,-2.05631,2.940627,-0.742176,-2.864411,-0.924307,1.106833,5.574050,4.752989,-0.205265
2,-0.404512,-0.809923,-0.788462,0.236565,-2.05631,0.132989,-0.755253,-2.864411,-0.924307,1.106833,5.574050,4.752989,-0.205265
3,-0.404512,-0.809923,-0.788462,0.236565,-2.05631,0.600928,-0.588406,-2.864411,-0.924307,1.106833,5.574050,4.752989,-0.205265
4,-0.404512,-0.809923,-0.788462,0.236565,-2.05631,-0.334951,-0.503685,-2.864411,-0.924307,1.106833,5.574050,4.752989,-0.205265
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31135,-0.404512,-0.809923,-0.788462,0.236565,-2.05631,-1.738770,-0.649500,-2.864411,-0.924307,-0.903478,-0.179403,-0.210394,4.871747
31136,-0.404512,-0.809923,-0.788462,0.236565,-2.05631,-1.738770,5.513456,-2.864411,-0.924307,-0.903478,-0.179403,-0.210394,4.871747
31137,-0.404512,-0.809923,-0.788462,0.236565,-2.05631,-1.738770,-0.649500,-2.864411,-0.924307,-0.903478,-0.179403,-0.210394,4.871747
31138,-0.404512,-0.809923,-0.788462,0.236565,-2.05631,-1.738770,4.715735,-2.864411,-0.924307,-0.903478,-0.179403,-0.210394,-0.205265


In [52]:
# loss_fct = torch.nn.MSELoss()

# test_loss, r_squared, actual_vals, predictions, baseline_loss = validate_trained_model(model, test_set_loader.dataset, loss_fct, device)
# print(f'Test Loss: {test_loss}')
# print(f'r_squared: {r_squared}')
# print(f'baseline_loss: {baseline_loss}')

# print(f'actual_vals shape: {len(actual_vals)}')
# print(f'predictions shape: {len(predictions)}')



In [53]:
# Function to filter list of tensors
# def filter_tensors_by_indices(tensor_list, indices):
#     return [tensor[indices] for tensor in tensor_list]

In [54]:
# Ensure actual_vals and predictions are tensors
# if isinstance(actual_vals, list):
#     actual_vals = torch.tensor(actual_vals)
# if isinstance(predictions, list):
#     predictions = torch.tensor(predictions)
# THIS IS FOR WHOLE TEST SET ! 


# # Filter actual and predicted values
# filtered_actual = filter_tensors_by_indices(actual_vals, indices_to_filter_for)
# filtered_predicted = filter_tensors_by_indices(predictions, indices_to_filter_for)

# # Concatenate filtered tensors for computing metrics
# filtered_actual_concat = torch.cat(filtered_actual)
# filtered_predicted_concat = torch.cat(filtered_predicted)

# filtered_actual_mean = torch.mean(filtered_actual_concat)
# # filtered_actual = actual_vals[indices_to_filter_for]
# # filtered_actual_mean = torch.mean(filtered_actual)
# # filtered_predicted = predictions[indices_to_filter_for]

# mse_filtered = loss_fct(filtered_actual_concat, filtered_predicted_concat)
# baseline_filtered = loss_fct(filtered_actual_concat, torch.full_like(filtered_actual_concat, filtered_actual_mean))
# r_squared = compute_r2_torch(preds=filtered_predicted_concat, targets=filtered_actual_concat)
# loss_fct_aux = torch.nn.MSELoss(reduction='none')
# variance = torch.var(loss_fct_aux(filtered_actual_concat, torch.full_like(filtered_actual_concat, filtered_actual_mean)))

# print(f'variance: {variance}')
# print(f'Baseline Loss: {baseline_filtered}')
# print(f'Test Loss: {mse_filtered}')
# print(f'r_squared: {r_squared}')


In [13]:
# # gdf = hf.data_to_geodataframe(data=test_set_loader.dataset[0], original_gdf=test_data, predicted_values=predictions)
# # gdf_with_og_values = hf.map_to_original_values(input_gdf=gdf, scaler_x=scaler_x)
# # gdf_with_og_values.crs = "EPSG:2154"
# # gdf_with_og_values.to_crs("EPSG:4326", inplace=True)

# gdf_in_meters = gdf_with_og_values.to_crs("EPSG:32633")
# gdf_in_meters.length

# tolerance = 1e-3
# gdf_with_capacity_reduction = gdf_in_meters.loc[indices_to_filter_for]
# gdf_with_capacity_reduction['length'] = gdf_with_capacity_reduction.length
# total_length = gdf_with_capacity_reduction['length'].sum()/1000
# print(total_length)

In [15]:
gdf_in_meters = gdf_with_og_values.to_crs("EPSG:32633")
gdf_in_meters.length

tolerance = 1e-3
gdf_with_capacity_reduction = gdf_in_meters[abs(gdf_in_meters['og_capacity_reduction']) > tolerance]
gdf_with_capacity_reduction['length'] = gdf_with_capacity_reduction.length
total_length = gdf_with_capacity_reduction['length'].sum()/1000

/opt/anaconda3/envs/chenhao-gnn/lib/python3.10/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [16]:
len(gdf_with_capacity_reduction)

4492